# Predicción de Cantidades Vendidas por Grupo (Prophet)

Este notebook replica el flujo de predicción de cantidades vendidas usando Prophet, pero ahora genera un modelo y predicción por cada grupo de `REGION_CATEGORIA_PRODUCTO`. Se exportan todas las predicciones y métricas a Snowflake, y se visualiza solo el 10% de los grupos seleccionados al azar.

## 1. Importar Librerías y Configuración

Importamos las librerías necesarias y configuramos logging y warnings.

In [ ]:
import pandas as pd
import numpy as np
import snowflake.connector
import logging
from datetime import datetime
import os
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
import plotly.subplots as sp
from tqdm import tqdm
import random
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)

# Suppress Prophet and related loggers
for logger_name in ['prophet', 'fbprophet', 'cmdstanpy']:
    logging.getLogger(logger_name).setLevel(logging.CRITICAL)

## 2. Carga de Datos desde Snowflake

Definimos funciones para conectar y cargar los dataframes históricos y de escenarios desde Snowflake, sin filtrar por REGION_CATEGORIA_PRODUCTO.

In [ ]:
def get_snowflake_connection():
    try:
        conn = snowflake.connector.connect(
            user='XXXXXXXXXXXX',
            password='XXXXXXXXXXXXXXXXX',
            account='XXXXXXXXXXXXX',
            warehouse='COMPUTE_WH',
            database='BEBIDAS_PROJECT',
            schema='BEBIDAS_ANALYTICS'
        )
        logging.info("Conexión a Snowflake exitosa")
        return conn
    except Exception as e:
        logging.error(f"Error de conexión a Snowflake: {e}")
        raise

def load_snowflake_views():
    required_cols = ['MES', 'REGION_CATEGORIA_PRODUCTO',  'AVG_PRECIO_LISTA', 'AVG_DESC_PORCENTAJE']
    conn = get_snowflake_connection()
    try:
        df_hist = pd.read_sql("SELECT * FROM VW_VENTAS_HISTORICO_M3", conn)
        df_fcst1 = pd.read_sql("SELECT * FROM VW_VENTAS_FCST_FEATURES_1", conn)
        df_fcst2 = pd.read_sql("SELECT * FROM VW_VENTAS_FCST_FEATURES_2", conn)
        
        # Validate columns
        for df, name in [(df_hist, 'Histórico'), (df_fcst1, 'Escenario 1'), (df_fcst2, 'Escenario 2')]:
            missing_cols = [col for col in required_cols if col not in df.columns]
            if missing_cols:
                raise ValueError(f"Missing columns in {name}: {missing_cols}")
        
        logging.info(f"Histórico: {len(df_hist)} registros, Escenario 1: {len(df_fcst1)}, Escenario 2: {len(df_fcst2)}")
        return df_hist, df_fcst1, df_fcst2
    finally:
        conn.close()

# Load data
df_hist, df_fcst1, df_fcst2 = load_snowflake_views()
print(f"Histórico: {df_hist.shape}, Escenario 1: {df_fcst1.shape}, Escenario 2: {df_fcst2.shape}")

2025-06-29 00:42:19,526 - INFO - Histórico: 3703 registros, Escenario 1: 1560, Escenario 2: 1560


Histórico: (3703, 6), Escenario 1: (1560, 5), Escenario 2: (1560, 5)


## 3. Preparación de Datos para Prophet por Grupo

Identificamos todos los valores únicos de `REGION_CATEGORIA_PRODUCTO`. Para cada grupo, preparamos los dataframes históricos y de escenarios, escalando variables y agregando columnas requeridas para Prophet.

In [12]:
# Exogenous variables
exog_vars = ['AVG_PRECIO_LISTA', 'AVG_DESC_PORCENTAJE']

# Identify unique groups
grupos = df_hist['REGION_CATEGORIA_PRODUCTO'].unique()
print(f"Total de grupos: {len(grupos)}")

# Initialize result and metrics lists
resultados_todos = []
metricas_todos = []

Total de grupos: 130


## 4. Entrenamiento y Predicción por Grupo

Iteramos sobre cada grupo. Para cada uno, entrenamos un modelo Prophet con variables exógenas, generamos predicciones para ambos escenarios, desescalamos resultados y calculamos intervalos de confianza.

In [13]:
from tqdm import tqdm

# Clear lists to avoid duplicates
resultados_todos = []
metricas_todos = []

for grupo in tqdm(grupos, desc="Procesando grupos Prophet"):
    # Filter data by group
    hist_g = df_hist[df_hist['REGION_CATEGORIA_PRODUCTO'] == grupo].copy()
    fcst1_g = df_fcst1[df_fcst1['REGION_CATEGORIA_PRODUCTO'] == grupo].copy()
    fcst2_g = df_fcst2[df_fcst2['REGION_CATEGORIA_PRODUCTO'] == grupo].copy()
    
    # Skip groups with insufficient data
    if len(hist_g) < 12 or len(fcst1_g) == 0 or len(fcst2_g) == 0:
        logging.warning(f"Grupo {grupo} omitido: pocos datos (Hist: {len(hist_g)}, Fcst1: {len(fcst1_g)}, Fcst2: {len(fcst2_g)})")
        continue
    
    # Prepare columns
    hist_g['ds'] = pd.to_datetime(hist_g['MES'])
    hist_g['y'] = hist_g['M3_VENDIDOS']
    fcst1_g['ds'] = pd.to_datetime(fcst1_g['MES'])
    fcst2_g['ds'] = pd.to_datetime(fcst2_g['MES'])
    
    # Scale variables
    target_scaler = StandardScaler()
    exog_scaler = StandardScaler()
    hist_g['y_scaled'] = target_scaler.fit_transform(hist_g[['y']]).flatten()
    scaled_exog = exog_scaler.fit_transform(hist_g[exog_vars])
    for idx, var in enumerate(exog_vars):
        hist_g[f'{var}_scaled'] = scaled_exog[:, idx]
    
    def scale_exog(df):
        scaled = exog_scaler.transform(df[exog_vars])
        for idx, var in enumerate(exog_vars):
            df[f'{var}_scaled'] = scaled[:, idx]
        return df
    
    fcst1_g = scale_exog(fcst1_g)
    fcst2_g = scale_exog(fcst2_g)
    exog_vars_scaled = [f'{var}_scaled' for var in exog_vars]
    
    # Train Prophet model
    m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
    for var in exog_vars_scaled:
        m.add_regressor(var)
    m.fit(hist_g[['ds', 'y_scaled'] + exog_vars_scaled].rename(columns={'y_scaled': 'y'}))
    
    # Prepare future dataframes
    def prepare_future_scaled(df_fcst):
        return df_fcst[['ds'] + exog_vars_scaled].copy()
    
    future1 = prepare_future_scaled(fcst1_g)
    future2 = prepare_future_scaled(fcst2_g)
    
    # Generate predictions
    forecast1 = m.predict(future1)
    forecast2 = m.predict(future2)
    
    # Inverse scale predictions
    fcst1_g['CANTIDAD_VENDIDA_PRED'] = target_scaler.inverse_transform(forecast1[['yhat']]).flatten()
    fcst1_g['IC_95_LOWER'] = target_scaler.inverse_transform(forecast1[['yhat_lower']]).flatten()
    fcst1_g['IC_95_UPPER'] = target_scaler.inverse_transform(forecast1[['yhat_upper']]).flatten()
    fcst2_g['CANTIDAD_VENDIDA_PRED'] = target_scaler.inverse_transform(forecast2[['yhat']]).flatten()
    fcst2_g['IC_95_LOWER'] = target_scaler.inverse_transform(forecast2[['yhat_lower']]).flatten()
    fcst2_g['IC_95_UPPER'] = target_scaler.inverse_transform(forecast2[['yhat_upper']]).flatten()
    
    # Add metadata
    hist_g['TIPO'] = 'HISTÓRICO'
    hist_g['IC_95_LOWER'] = np.nan
    hist_g['IC_95_UPPER'] = np.nan
    hist_g['DIF_PCT'] = hist_g['y'].pct_change() * 100
    fcst1_g['TIPO'] = 'ESCENARIO 1'
    fcst2_g['TIPO'] = 'ESCENARIO 2'
    
    # Calculate percentage differences
    last_hist = hist_g['y'].iloc[-1]
    def calc_diff(df, col, first_ref=None):
        diffs = df[col].pct_change() * 100
        if first_ref is not None and len(diffs) > 0:
            diffs.iloc[0] = 100 * (df[col].iloc[0] - first_ref) / ((abs(df[col].iloc[0]) + abs(first_ref)) / 2)
        return diffs
    
    fcst1_g['DIF_PCT'] = calc_diff(fcst1_g, 'CANTIDAD_VENDIDA_PRED', first_ref=last_hist)
    fcst2_g['DIF_PCT'] = calc_diff(fcst2_g, 'CANTIDAD_VENDIDA_PRED', first_ref=last_hist)
    
    # Consolidate results
    hist_tabla = hist_g[['ds', 'REGION_CATEGORIA_PRODUCTO', 'y', 'TIPO', 'IC_95_LOWER', 'IC_95_UPPER', 'DIF_PCT']].rename(columns={'y': 'M3_PRODUCTO'})
    sc1 = fcst1_g[['ds', 'REGION_CATEGORIA_PRODUCTO', 'CANTIDAD_VENDIDA_PRED', 'TIPO', 'IC_95_LOWER', 'IC_95_UPPER', 'DIF_PCT']].rename(columns={'CANTIDAD_VENDIDA_PRED': 'M3_PRODUCTO'})
    sc2 = fcst2_g[['ds', 'REGION_CATEGORIA_PRODUCTO', 'CANTIDAD_VENDIDA_PRED', 'TIPO', 'IC_95_LOWER', 'IC_95_UPPER', 'DIF_PCT']].rename(columns={'CANTIDAD_VENDIDA_PRED': 'M3_PRODUCTO'})
    
    # Combine and append results
    resultados_todos.append(pd.concat([hist_tabla, sc1, sc2], ignore_index=True))
    
    def calcular_metricas(hist_g, m, target_scaler, exog_vars_scaled, grupo, metricas_todos, exog_vars, exog_scaler):
        def smape(y_true, y_pred):
            denominator = np.abs(y_true) + np.abs(y_pred)
            # Use np.where instead of .where for numpy arrays
            denominator_safe = np.where(denominator != 0, denominator, 1)
            return 100 * np.mean(2 * np.abs(y_pred - y_true) / denominator_safe)
        
        # Use the last 7 rows or all if fewer than 7
        n_test = min(7, len(hist_g))
        test_hist = hist_g.tail(n_test).copy()
        
        # Prepare future dataframe for prediction
        future_test = test_hist[['ds'] + exog_vars_scaled]
        forecast_test = m.predict(future_test)
        
        # Inverse scale predictions
        y_true = target_scaler.inverse_transform(test_hist[['y_scaled']]).flatten()
        y_pred = target_scaler.inverse_transform(forecast_test[['yhat']]).flatten()
        
        # Calculate metrics
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        mae = mean_absolute_error(y_true, y_pred)
        smape_val = smape(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        
        # Determine model quality
        if r2 >= 0.8 and smape_val < 20:
            quality = 'BUENO'
        elif r2 >= 0.5 or smape_val < 30:
            quality = 'ACEPTABLE'
        else:
            quality = 'POBRE'
        
        # Append metrics
        metricas_todos.append({
            'REGION_CATEGORIA_PRODUCTO': grupo,
            'RMSE': rmse,
            'MAE': mae,
            'SMAPE': smape_val,
            'R2': r2,
            'QUALITY': quality
        })
        
    # Calculate metrics
    calcular_metricas(hist_g, m, target_scaler, exog_vars_scaled, grupo, metricas_todos, exog_vars, exog_scaler)

Procesando grupos Prophet: 100%|██████████| 130/130 [04:16<00:00,  1.98s/it]


In [14]:
print(f"Grupos procesados: {len(resultados_todos)}")
if len(resultados_todos) != len(grupos):
    logging.warning(f"Se esperaban {len(grupos)} grupos, pero se procesaron {len(resultados_todos)}")

Grupos procesados: 130


## 5. Cálculo de Métricas por Grupo

Para cada grupo, calculamos métricas de desempeño (RMSE, MAE, SMAPE, R2) en el histórico reciente y determinamos la calidad del modelo.

In [15]:

df_metricas = pd.DataFrame(metricas_todos)
print(f"Total de grupos con métricas: {len(df_metricas)}")
if len(df_metricas) != len(grupos):
    logging.warning(f"Se esperaban {len(grupos)} métricas, pero se generaron {len(df_metricas)}")

# Display metrics
display(df_metricas)
display(df_metricas.describe())

Total de grupos con métricas: 130


,REGION_CATEGORIA_PRODUCTO,RMSE,MAE,SMAPE,R2,QUALITY
0,Antioquia-Jugo-Jugo Lima 2000mL x 12uds,2.572263,2.354717,2.851329,0.888603,BUENO
1,Antioquia-Gaseosa-Piña 600mL x 12uds,1.719893,1.518880,8.643584,0.538679,ACEPTABLE
2,Antioquia-Agua-Agua Sin Gas 1L x 1uds,0.112490,0.093440,2.599017,0.941358,BUENO
3,Antioquia-Jugo-Jugo Manzana 2000mL x 24uds,13.559341,10.685055,6.378865,0.868902,BUENO
4,Antioquia-Gaseosa-Uva 600mL x 24uds,3.283661,2.882006,5.806735,0.510225,ACEPTABLE
...,...,...,...,...,...,...
125,Valle del Cauca-Agua-Agua con Gas 5L x 1uds,1.875240,1.524013,14.142300,0.195414,ACEPTABLE
126,Valle del Cauca-Gaseosa-Piña 600mL x 12uds,0.789733,0.726848,5.814025,0.915082,BUENO
127,Valle del Cauca-Bebida Energética-Energía Extr...,0.141058,0.112073,3.473786,0.909005,BUENO
128,Boyacá-Gaseosa-Piña 600mL x 12uds,0.000010,0.000008,0.019594,1.000000,BUENO


,RMSE,MAE,SMAPE,R2
count,130.000000,130.000000,130.000000,130.000000
mean,2.063629,1.639999,12.868330,0.676229
std,3.075820,2.442266,9.083117,0.290612
min,0.000002,0.000002,0.000443,-0.779869
25%,0.141411,0.100874,6.112180,0.564435
50%,0.613346,0.489698,11.007667,0.758302
75%,2.893601,2.272756,16.551181,0.868423
max,13.559341,10.787754,45.985436,1.000000


## 7. Visualización Aleatoria del 10% de los Grupos

Seleccionamos aleatoriamente el 10% de los grupos y graficamos para cada uno: evolución de variables exógenas, resultados de predicción, diferencias porcentuales mes a mes y tendencias.

In [16]:
# Seleccionar 10% de los grupos al azar para visualización
grupos_sample = random.sample(list(grupos), max(1, int(0.1 * len(grupos))))
print(f"Grupos seleccionados para visualización: {grupos_sample}")

# Create df_resultados from resultados_todos
df_resultados = pd.concat(resultados_todos, ignore_index=True) if resultados_todos else pd.DataFrame()

import re

def hex_to_rgba(hex_color, alpha=0.2):
    """Convert hex color string to rgba string with given alpha."""
    hex_color = hex_color.lstrip('#')
    if len(hex_color) == 6:
        r, g, b = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
        return f'rgba({r},{g},{b},{alpha})'
    else:
        # fallback to gray if invalid
        return f'rgba(128,128,128,{alpha})'

for grupo in grupos_sample:
    df_g = df_resultados[df_resultados['REGION_CATEGORIA_PRODUCTO'] == grupo].copy()
    if df_g.empty:
        logging.warning(f"No hay datos para el grupo {grupo} en df_resultados")
        continue

    # Evolución de variables exógenas con Plotly
    fig_exog = sp.make_subplots(rows=len(exog_vars), cols=1, 
                                subplot_titles=[f'{grupo} - Evolución de {var}' for var in exog_vars],
                                shared_xaxes=True, vertical_spacing=0.1)

    for i, var in enumerate(exog_vars, start=1):
        hist_g = df_hist[df_hist['REGION_CATEGORIA_PRODUCTO'] == grupo].copy()
        fcst1_g = df_fcst1[df_fcst1['REGION_CATEGORIA_PRODUCTO'] == grupo].copy()
        fcst2_g = df_fcst2[df_fcst2['REGION_CATEGORIA_PRODUCTO'] == grupo].copy()
        
        # Add traces for each dataset
        fig_exog.add_trace(
            go.Scatter(x=hist_g['MES'], y=hist_g[var], mode='lines', name='Histórico', 
                        line=dict(color='black')),
            row=i, col=1
        )
        fig_exog.add_trace(
            go.Scatter(x=fcst1_g['MES'], y=fcst1_g[var], mode='lines', name='Escenario 1', 
                        line=dict(color='#8B7557')),  
            row=i, col=1
        )
        fig_exog.add_trace(
            go.Scatter(x=fcst2_g['MES'], y=fcst2_g[var], mode='lines', name='Escenario 2', 
                        line=dict(color='#1f77b4')),  
            row=i, col=1
        )

    # Update layout
    fig_exog.update_layout(
        height=300 * len(exog_vars), width=1000,
        title_text=f'Evolución de Variables Exógenas - {grupo}',
        showlegend=True,
                legend_title='Serie',
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=-0.3,
            x=0.5,
            xanchor='center')
    )
    fig_exog.update_xaxes(title_text='Fecha', row=len(exog_vars), col=1)
    for i, var in enumerate(exog_vars, start=1):
        fig_exog.update_yaxes(title_text=var, row=i, col=1)
    fig_exog.show()

    # Resultados de predicción con intervalos de confianza
    fig = go.Figure()
    color_list = ['#36332E', '#8B7557', '#57788B']
    for tipo, color in zip(['HISTÓRICO', 'ESCENARIO 1', 'ESCENARIO 2'], color_list):
        df_tipo = df_g[df_g['TIPO'] == tipo]
        # Add main prediction line
        fig.add_trace(go.Scatter(
            x=df_tipo['ds'], 
            y=df_tipo['M3_PRODUCTO'], 
            mode='lines', 
            name=tipo, 
            line=dict(color=color)
        ))
        # Add confidence intervals for forecast scenarios only
        if tipo in ['ESCENARIO 1', 'ESCENARIO 2']:
            # Upper bound
            fig.add_trace(go.Scatter(
                x=df_tipo['ds'], 
                y=df_tipo['IC_95_UPPER'], 
                mode='lines', 
                line=dict(width=0), 
                showlegend=False, 
                hoverinfo='skip'
            ))
            # Lower bound with fill to upper bound
            fig.add_trace(go.Scatter(
                x=df_tipo['ds'], 
                y=df_tipo['IC_95_LOWER'], 
                mode='lines', 
                line=dict(width=0), 
                fill='tonexty', 
                fillcolor=hex_to_rgba(color, 0.2),  # Proper rgba color
                showlegend=False, 
                hoverinfo='skip'
            ))

    fig.update_layout(
        title=f'Predicción de Cantidades Vendidas - {grupo}', 
        xaxis_title='Fecha', 
        yaxis_title='Cantidad Vendida (m3)', 
        legend_title='Serie',
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=-0.35,
            x=0.5,
            xanchor='center'),
        width=1000, 
        height=400
    )
    fig.show()

    # Diferencias porcentuales
    fig = go.Figure()
    for tipo, color in zip(['HISTÓRICO', 'ESCENARIO 1', 'ESCENARIO 2'], ['#36332E', '#8B7557', '#57788B']):
        df_tipo = df_g[df_g['TIPO'] == tipo]
        fig.add_trace(go.Bar(x=df_tipo['ds'], y=df_tipo['DIF_PCT'], name=tipo, marker_color=color))
    fig.update_layout(
        title=f'Diferencia porcentual mes a mes - {grupo}', 
        xaxis_title='Fecha', 
        yaxis_title='% Dif.',
        legend_title='Serie',
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=-0.35,
            x=0.5,
            xanchor='center'), 
        width=1000, 
        height=400
    )
    fig.show()

Grupos seleccionados para visualización: ['Norte de Santander-Jugo-Jugo Lima 2000mL x 12uds', 'Boyacá-Jugo-Jugo Manzana 1000mL x 24uds', 'Santander-Bebida de Té-Té Verde 300mL x 1uds', 'Atlántico-Gaseosa-Piña 600mL x 12uds', 'Atlántico-Bebida Energética-Power Up 250mL x 4uds', 'Tolima-Bebida Energética-Power Up 250mL x 4uds', 'Bolívar-Agua-Agua Sin Gas 5L x 12uds', 'Valle del Cauca-Jugo-Jugo Lima 2000mL x 12uds', 'Meta-Bebida Energética-Power Up 250mL x 4uds', 'Norte de Santander-Bebida de Té-Té Verde 300mL x 1uds', 'Bolívar-Jugo-Jugo Manzana 2000mL x 24uds', 'Tolima-Jugo-Jugo Manzana 1000mL x 24uds', 'Cundinamarca-Gaseosa-Piña 600mL x 12uds']



## 6. Consolidación y Exportación de Resultados a Snowflake

Unificamos los resultados y métricas de todos los grupos en dataframes finales y exportamos a Snowflake usando COPY INTO e INSERT INTO.

In [17]:
# Consolidate results
df_resultados = pd.concat(resultados_todos, ignore_index=True) if resultados_todos else pd.DataFrame()

# Add split columns
split_cols = df_resultados['REGION_CATEGORIA_PRODUCTO'].str.split('-', n=2, expand=True)
df_resultados['REGION'] = split_cols[0]
df_resultados['CATEGORIA'] = split_cols[1]
df_resultados['PRODUCTO'] = split_cols[2]
df_resultados = df_resultados.rename(columns={'ds': 'FECHA'})

# Validate row count
n_hist = len(df_hist)
n_fcst1 = len(df_fcst1)
n_fcst2 = len(df_fcst2)
suma_esperada = n_hist + n_fcst1 + n_fcst2
suma_real = len(df_resultados)
print(f"Filas esperadas (histórico + esc1 + esc2): {suma_esperada}")
print(f"Filas reales en df_resultados: {suma_real}")
if suma_esperada == suma_real:
    print("✔️ La suma de filas es correcta.")
else:
    logging.warning(f"❌ Diferencia detectada: esperadas {suma_esperada}, reales {suma_real}")

Filas esperadas (histórico + esc1 + esc2): 6823
Filas reales en df_resultados: 6823
✔️ La suma de filas es correcta.


In [18]:

# Exportar a Snowflake (ajustar según tu función de exportación)
def export_prophet_results_to_snowflake(df_resultados, df_metricas):
    conn = get_snowflake_connection()
    cursor = conn.cursor()
    timestamp_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    temp_csv_path = 'temp_prophet_resultados.csv'
    temp_metrics_path = 'temp_prophet_metrics.csv'
    try:
        # Crear tablas si no existen
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS PROPHET_M3_RESULTADOS (
                ID INTEGER AUTOINCREMENT,
                TIMESTAMP_COL TIMESTAMP_NTZ,
                REGION_CATEGORIA_PRODUCTO VARCHAR(300),
                REGION VARCHAR(100),
                CATEGORIA VARCHAR(100),
                PRODUCTO VARCHAR(100),
                FECHA DATE,
                TIPO VARCHAR(30),
                M3_PRODUCTO FLOAT,
                IC_95_LOWER FLOAT,
                IC_95_UPPER FLOAT,
                DIF_PCT FLOAT
            )
        """)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS PROPHET_M3_METRICS (
                ID INTEGER AUTOINCREMENT,
                TIMESTAMP_COL TIMESTAMP_NTZ,
                REGION_CATEGORIA_PRODUCTO VARCHAR(300),
                RMSE FLOAT,
                MAE FLOAT,
                SMAPE FLOAT,
                R2 FLOAT,
                QUALITY VARCHAR(20)
            )
        """)
        # Exportar resultados
        df_resultados['TIMESTAMP_COL'] = timestamp_str
        cols_result = ['TIMESTAMP_COL', 'REGION_CATEGORIA_PRODUCTO', 'REGION', 'CATEGORIA', 'PRODUCTO', 'FECHA', 'TIPO', 'M3_PRODUCTO', 'IC_95_LOWER', 'IC_95_UPPER', 'DIF_PCT']
        df_resultados[cols_result].to_csv(temp_csv_path, index=False, header=False, encoding='utf-8')
        cursor.execute(f"PUT file://{temp_csv_path} @~/prophet_stage AUTO_COMPRESS = TRUE OVERWRITE = TRUE")
        cursor.execute("TRUNCATE TABLE PROPHET_M3_RESULTADOS")
        cursor.execute(f"""
            COPY INTO PROPHET_M3_RESULTADOS (TIMESTAMP_COL, REGION_CATEGORIA_PRODUCTO, REGION, CATEGORIA, PRODUCTO, FECHA, TIPO, M3_PRODUCTO, IC_95_LOWER, IC_95_UPPER, DIF_PCT)
            FROM @~/prophet_stage/{temp_csv_path}.gz
            FILE_FORMAT = (TYPE = 'CSV' FIELD_DELIMITER = ',' SKIP_HEADER = 0 FIELD_OPTIONALLY_ENCLOSED_BY = '"')
        """)
        # Exportar métricas
        df_metricas['TIMESTAMP_COL'] = timestamp_str
        cols_metrics = ['TIMESTAMP_COL', 'REGION_CATEGORIA_PRODUCTO', 'RMSE', 'MAE', 'SMAPE', 'R2', 'QUALITY']
        df_metricas[cols_metrics].to_csv(temp_metrics_path, index=False, header=False, encoding='utf-8')
        cursor.execute(f"PUT file://{temp_metrics_path} @~/prophet_stage AUTO_COMPRESS = TRUE OVERWRITE = TRUE")
        cursor.execute(f"""
            COPY INTO PROPHET_M3_METRICS (TIMESTAMP_COL, REGION_CATEGORIA_PRODUCTO, RMSE, MAE, SMAPE, R2, QUALITY)
            FROM @~/prophet_stage/{temp_metrics_path}.gz
            FILE_FORMAT = (TYPE = 'CSV' FIELD_DELIMITER = ',' SKIP_HEADER = 0 FIELD_OPTIONALLY_ENCLOSED_BY = '"')
        """)
        conn.commit()
        print('Exportación a Snowflake completada.')
    except Exception as e:
        conn.rollback()
        print(f'Error al exportar a Snowflake: {e}')
    finally:
        cursor.close()
        conn.close()
        import os
        if os.path.exists(temp_csv_path):
            os.remove(temp_csv_path)
        if os.path.exists(temp_metrics_path):
            os.remove(temp_metrics_path)

# Llamar exportación
export_prophet_results_to_snowflake(df_resultados, df_metricas)

def print_snowflake_counts():
    try:
        conn = get_snowflake_connection()
        cursor = conn.cursor()
        cursor.execute('SELECT COUNT(*) FROM PROPHET_M3_RESULTADOS')
        count_resultados = cursor.fetchone()[0]
        print(f'Número de Registros: {count_resultados}')
        cursor.execute('SELECT COUNT(*) FROM PROPHET_M3_METRICS')
        count_metrics = cursor.fetchone()[0]
        print(f'Número de Registros: {count_metrics}')
        cursor.close()
        conn.close()
    except Exception as e:
        print(f'Error al consultar tablas en Snowflake: {e}')

# Llamar solo una vez para evitar duplicados
print_snowflake_counts()

2025-06-29 00:46:42,488 - INFO - Snowflake Connector for Python Version: 3.15.0, Python Version: 3.10.11, Platform: Windows-10-10.0.19045-SP0
2025-06-29 00:46:42,493 - INFO - Connecting to GLOBAL Snowflake domain
2025-06-29 00:46:42,493 - INFO - Connecting to GLOBAL Snowflake domain
2025-06-29 00:46:43,523 - INFO - Conexión a Snowflake exitosa
2025-06-29 00:46:43,523 - INFO - Conexión a Snowflake exitosa


Exportación a Snowflake completada.


2025-06-29 00:46:49,891 - INFO - Snowflake Connector for Python Version: 3.15.0, Python Version: 3.10.11, Platform: Windows-10-10.0.19045-SP0
2025-06-29 00:46:49,894 - INFO - Connecting to GLOBAL Snowflake domain
2025-06-29 00:46:49,894 - INFO - Connecting to GLOBAL Snowflake domain
2025-06-29 00:46:50,530 - INFO - Conexión a Snowflake exitosa
2025-06-29 00:46:50,530 - INFO - Conexión a Snowflake exitosa


Número de Registros: 6823
Número de Registros: 412
Número de Registros: 412
